In [14]:
import os
import keras
import random
import pathlib
import numpy as np
import tensorflow as tf


from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

In [15]:
# Memory error, so to feed model in batches
# Image transformer
batch_size = 20
train_dir = '../Capstone/few_images/'

def custom_preprocessing(image):
    state = random.randint(0,2)
    if state == 0:
        processed_img = exposure.equalize_adapthist((image*1.0/255), clip_limit=0.02)
    elif state == 1:
        processed_img = exposure.equalize_hist(image)
    elif state == 2:
        p2, p98 = np.percentile(image, (2,98))
        processed_img = exposure.rescale_intensity(image, in_range=(p2,p98))
    return processed_img

image_gen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        vertical_flip=True,
        preprocessing_function=custom_preprocessing)

# function to split training set X train, y train and produce augmented images       
def image_a_b_gen(batch_size):
    for i in image_gen.flow_from_directory(train_dir, batch_size=batch_size,class_mode=None,shuffle=False):
        lab_batch = rgb2lab(i)
        X_train = lab_batch[:,:,:,0]
        X_train = X_train.reshape(X_train.shape+(1,))
        y_train = lab_batch[:,:,:,1:] / 128
        yield ([X_train, y_train])


In [17]:
class DSSIMObjective:
    def __init__(self, k1=0.01, k2=0.03, max_value=1.0):
        self.__name__ = 'DSSIMObjective'
        self.k1 = k1
        self.k2 = k2
        self.max_value = max_value
        self.backend = K.backend()

    def __int_shape(self, x):
        return K.int_shape(x) if self.backend == 'tensorflow' else K.shape(x)

    def __call__(self, y_true, y_pred):
        ch = K.shape(y_pred)[-1]

        def _fspecial_gauss(size, sigma):
            #Function to mimic the 'fspecial' gaussian MATLAB function.
            coords = np.arange(0, size, dtype=K.floatx())
            coords -= (size - 1 ) / 2.0
            g = coords**2
            g *= ( -0.5 / (sigma**2) )
            g = np.reshape (g, (1,-1)) + np.reshape(g, (-1,1) )
            g = K.constant ( np.reshape (g, (1,-1)) )
            g = K.softmax(g)
            g = K.reshape (g, (size, size, 1, 1)) 
            g = K.tile (g, (1,1,ch,1))
            return g
                  
        kernel = _fspecial_gauss(11,1.5)

        def reducer(x):
            return K.depthwise_conv2d(x, kernel, strides=(1, 1), padding='valid')

        c1 = (self.k1 * self.max_value) ** 2
        c2 = (self.k2 * self.max_value) ** 2
        
        mean0 = reducer(y_true)
        mean1 = reducer(y_pred)
        num0 = mean0 * mean1 * 2.0
        den0 = K.square(mean0) + K.square(mean1)
        luminance = (num0 + c1) / (den0 + c1)
        
        num1 = reducer(y_true * y_pred) * 2.0
        den1 = reducer(K.square(y_true) + K.square(y_pred))
        c2 *= 1.0 #compensation factor
        cs = (num1 - num0 + c2) / (den1 - den0 + c2)

        ssim_val = K.mean(luminance * cs, axis=(-3, -2) )
        return K.mean( (1.0 - ssim_val ) / 2.0 )

In [18]:
ssim_loss = DSSIMObjective()

In [16]:
#Shared models
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = BatchNormalization()(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = BatchNormalization()(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output_shared = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)

#Model A
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output_shared)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Model B
global_encoder = Conv2D(512, (3,3), activation='relu', padding='same',strides=2)(encoder_output_shared)
global_encoder = Conv2D(512, (3,3), activation='relu', padding='same')(global_encoder)
global_encoder = BatchNormalization()(global_encoder)
global_encoder = Conv2D(512, (3,3), activation='relu', padding='same',strides=2)(global_encoder)
global_encoder = Conv2D(512, (3,3), activation='relu', padding='same')(global_encoder)
global_encoder = BatchNormalization()(global_encoder)
global_encoder = Flatten()(global_encoder)
global_encoder = Dense(1024, activation='relu')(global_encoder)
global_encoder = Dense(512, activation='relu')(global_encoder)
global_encoder = Dense(256, activation='relu')(global_encoder)
global_encoder = RepeatVector(32 * 32)(global_encoder)
global_encoder = Reshape([32,32,256])(global_encoder)
#Fusion 
fusion_output = concatenate([encoder_output, global_encoder], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encoder_input, outputs=decoder_output)
# Finish model
model.compile(optimizer='adam',loss=ssim_loss ,metrics=['mse','mean_absolute_error'])

In [17]:
# Train model
model.fit_generator(image_a_b_gen(batch_size=batch_size), steps_per_epoch=10, epochs=1000)

Epoch 1/1000
Found 200 images belonging to 1 classes.
10/10 [==============================] - 193s 19s/step - loss: 0.8701
Epoch 2/1000
10/10 [==============================] - 203s 20s/step - loss: 0.9562
Epoch 3/1000
10/10 [==============================] - 209s 21s/step - loss: 0.9549
Epoch 4/1000
10/10 [==============================] - 203s 20s/step - loss: 0.9558
Epoch 5/1000
10/10 [==============================] - 202s 20s/step - loss: 0.9558
Epoch 6/1000
10/10 [==============================] - 205s 21s/step - loss: 0.9561
Epoch 7/1000
 2/10 [=====>........................] - ETA: 2:49 - loss: 0.9344

KeyboardInterrupt: 

In [14]:
model.save('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/3rd_expt_2.h5')

In [19]:
# Load black and white images
color_me = []
for filename in os.listdir('../Capstone/test'):
        color_me.append(img_to_array(load_img('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(color_me/255.0)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [20]:
# Test model
output = model.predict(color_me)
output = output * 128


In [22]:
# Output colorizations
for i in range(len(output)):
        cur = np.zeros((256, 256, 3))
        cur[:,:,0] = color_me[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave("C:/Users/n3rDx/Desktop/Homework Upload/Capstone/result/"+str(i)+".jpg", lab2rgb(cur))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 41429 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 64067 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\n3rDx\Anaconda3\lib\site-packages\skimage\color\colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 46016 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
Lossy conversion from 

In [29]:
color_me

array([[[[32.7475089 ],
         [30.59331625],
         [31.02611513],
         ...,
         [31.02611513],
         [30.59331625],
         [30.59331625]],

        [[32.7475089 ],
         [30.15950997],
         [29.72468203],
         ...,
         [45.62868914],
         [45.62868914],
         [45.62868914]],

        [[32.7475089 ],
         [30.15950997],
         [28.8519024 ],
         ...,
         [53.58501345],
         [53.97669749],
         [53.58501345]],

        ...,

        [[39.48966033],
         [39.48966033],
         [41.14266653],
         ...,
         [10.76705433],
         [10.76705433],
         [ 9.26323429]],

        [[39.48966033],
         [39.90405327],
         [41.14266653],
         ...,
         [11.26361052],
         [ 9.76693423],
         [ 8.75701082]],

        [[39.48966033],
         [39.90405327],
         [41.14266653],
         ...,
         [10.26818431],
         [ 9.26323429],
         [ 9.26323429]]],


       [[[37.22507152],


In [31]:
for i in image_a_b_gen(2):
    print(i)

Found 200 images belonging to 2 classes.
[array([[[[ 7.72691649],
         [ 7.72691649],
         [ 7.72691649],
         ...,
         [23.00456399],
         [23.00456399],
         [23.00456399]],

        [[ 8.40682247],
         [ 8.40682247],
         [ 8.40682247],
         ...,
         [23.00456399],
         [23.00456399],
         [23.00456399]],

        [[ 9.73536928],
         [ 9.73536928],
         [ 9.73536928],
         ...,
         [23.00456399],
         [23.00456399],
         [23.00456399]],

        ...,

        [[15.9630319 ],
         [15.9630319 ],
         [15.9630319 ],
         ...,
         [85.65577349],
         [85.65577349],
         [85.65577349]],

        [[15.9630319 ],
         [15.9630319 ],
         [15.9630319 ],
         ...,
         [81.40877309],
         [81.40877309],
         [81.40877309]],

        [[15.9630319 ],
         [15.9630319 ],
         [15.9630319 ],
         ...,
         [76.69561843],
         [76.69561843],
         [

KeyboardInterrupt: 